# Conditional Random Field example for position tagging
By LongGang Pang


In [1]:
import os

In [14]:
def construct_dataset(kind="training"):
    with open("%s_sentence.csv"%kind, "r") as fin:
        sentences = fin.readlines()
    with open("%s_labels.csv"%kind, "r") as fin:
        labels = fin.readlines()
    #print(sentences[-1])
    #print(labels[-1])
    docs = []
    for i in range(len(sentences)):
        texts = []
        istart, iend, _ = labels[i].split('|||')
        istart = int(istart)
        iend = int(iend)
        for j, w in enumerate(sentences[i]):
            is_speaker = "F"
            if j in range(istart, iend):
                is_speaker = 'T'
            texts.append((w, is_speaker))
        docs.append(texts)
    return docs

In [15]:
docs = construct_dataset()

In [16]:
docs[0]

[('凤', 'F'),
 ('姐', 'F'),
 ('因', 'F'),
 ('问', 'F'),
 ('何', 'F'),
 ('事', 'F'),
 ('．', 'F'),
 ('凤', 'T'),
 ('姐', 'T'),
 ('道', 'F'),
 ('：', 'F'),
 ('\n', 'F')]

In [20]:
import nltk
from tqdm import tqdm_notebook
data = []
for i, doc in tqdm_notebook(enumerate(docs)):

    # Obtain the list of tokens in the document
    tokens = [t for t, label in doc]

    # Perform POS tagging
    tagged = nltk.pos_tag(tokens)

    # Take the word, POS tag, and its label
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])


In [21]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [22]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=False)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,
     
    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

In [30]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

In [41]:
y_pred[2]
X_test[2][1][1]

'word.lower=幻'

In [73]:
from IPython.display import HTML as html_print, display

def cstr(s, color='black'):
    #return "<text style=color:{}>{}</text>".format(color, s)
    return "<span style=\"color: #ff0000\">{}</span>".format(s)
    #return "**{}**".format(s)

#left, word, right = 'foo' , 'abc' , 'bar'
#html_print(cstr(' '.join([left, cstr(word, color='red'), right]), color='black') )
from IPython.display import Markdown

In [74]:
# Let's take a look at a random sample in the testing set
def check_result(i=12):
    res = []
    for j, label in enumerate(y_pred[i]):
        word = X_test[i][j][1].split("=")[1]
        if label == 'T':
            res.append(cstr(word, color='red'))
        elif label == 'F':
            res.append(word)
    display(Markdown(' '.join(res)))

check_result()

见 面 时 彼 此 悲 喜 交 接 ， 未 免 又 大 哭 一 阵 ， 后 又 致 喜 庆 之 词 ． 宝 玉 心 中 品 度 黛 玉 ， 越 发 出 落 的 超 逸 了 ． <span style="color: #ff0000">黛</span> <span style="color: #ff0000">玉</span> 又 带 了 许 多 书 籍 来 ， 忙 着 打 扫 卧 室 ， 安 插 器 具 ， 又 将 些 纸 笔 等 物 分 送 宝 钗 ， 迎 春 ， 宝 玉 等 人 ． <span style="color: #ff0000">宝</span> <span style="color: #ff0000">玉</span> 又 将 北 静 王 所 赠 й к 香 串 珍 重 取 出 来 ， 转 赠 黛 玉 ． <span style="color: #ff0000">门</span> <span style="color: #ff0000">子</span> 说 ： 


In [76]:
for i in range(100):
    check_result(i)

<span style="color: #ff0000">宝</span> <span style="color: #ff0000">玉</span> 进 来 回 道 ： 


<span style="color: #ff0000">凤</span> <span style="color: #ff0000">姐</span> 道 ： 


<span style="color: #ff0000">警</span> <span style="color: #ff0000">幻</span> 劝 道 ： 


<span style="color: #ff0000">茗</span> <span style="color: #ff0000">烟</span> 道 ： 


<span style="color: #ff0000">小</span> <span style="color: #ff0000">丫</span> <span style="color: #ff0000">头</span> 笑 道 ： 


...

那 袭 人 自 见 了 秦 钟 的 人 品 出 众 ， 心 中 似 有 所 失 ， 痴 了 半 日 ， 自 己 心 中 又 起 了 呆 意 ， 乃 自 思 道 ： 


<span style="color: #ff0000">贾</span> <span style="color: #ff0000">珍</span> 冷 笑 道 ： 


<span style="color: #ff0000">尤</span> <span style="color: #ff0000">氏</span> 笑 道 ： 


<span style="color: #ff0000">一</span> <span style="color: #ff0000">个</span> <span style="color: #ff0000">人</span> 道 ： 


<span style="color: #ff0000">宝</span> <span style="color: #ff0000">玉</span> 忙 答 道 ： 
